In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = self._get_image_paths()

    def _get_image_paths(self):
        image_paths = []
        for filename in os.listdir(self.root_dir):
            if filename.endswith(".jpg"):
                image_paths.append(os.path.join(self.root_dir, filename))
        return image_paths

    def _is_tampered(self, filename):
        return 'tamp' in filename

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = 1 if self._is_tampered(img_path) else 0  # 1 for tampered, 0 for real
        return img, label


In [3]:
root_dir = '/content/drive/MyDrive/MICC-F220/MICC-F220'
batch_size = 32

In [4]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the required input size of your model
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [5]:
# Create dataset
dataset = CustomDataset(root_dir, transform=transform)

In [6]:
# Split dataset into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)  # Split training data further for validation

In [7]:
print("Size of train_data:", len(train_data))
print("Size of validation_data:", len(val_data))
print("Size of test_data:", len(test_data))

Size of train_data: 176
Size of validation_data: 22
Size of test_data: 22


In [8]:
# Create dataloaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [9]:
model = torchvision.models.densenet121(pretrained=True)
model.fc = torch.nn.Linear(model.classifier.in_features, 2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 98.2MB/s]


In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Example usage:
num_params = count_parameters(model)
print(f"Number of parameters in the model: {num_params}")

Number of parameters in the model: 7980906


In [11]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Define your loss function
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [13]:
# Define the device to run the model on (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model.to(device)

In [15]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the parameter gradients

            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

            running_loss += loss.item() * inputs.size(0)

        # Calculate average training loss for the epoch
        epoch_loss = running_loss / len(train_loader.dataset)

        # Evaluate the model on the validation set
        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)

        # Print training and validation statistics for the epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, "f"Train Loss: {epoch_loss:.4f}, "f"Validation Loss: {val_loss:.4f}, "
             f"Validation Accuracy: {val_accuracy:.4f}")

In [16]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss

            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

            running_loss += loss.item() * inputs.size(0)

    # Calculate average loss and accuracy
    avg_loss = running_loss / len(dataloader.dataset)
    accuracy = correct_predictions / total_predictions

    return avg_loss, accuracy


In [17]:
train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10)

Epoch 1/10, Train Loss: 5.0821, Validation Loss: 6.2301, Validation Accuracy: 0.2273
Epoch 2/10, Train Loss: 0.8021, Validation Loss: 6.5066, Validation Accuracy: 0.7273
Epoch 3/10, Train Loss: 0.2749, Validation Loss: 2.1759, Validation Accuracy: 0.8182
Epoch 4/10, Train Loss: 0.1723, Validation Loss: 5.3452, Validation Accuracy: 0.9091
Epoch 5/10, Train Loss: 0.2239, Validation Loss: 1.8093, Validation Accuracy: 0.9545
Epoch 6/10, Train Loss: 0.2161, Validation Loss: 1.4419, Validation Accuracy: 0.9545
Epoch 7/10, Train Loss: 0.0926, Validation Loss: 1.2580, Validation Accuracy: 0.9091
Epoch 8/10, Train Loss: 0.1766, Validation Loss: 1.1549, Validation Accuracy: 0.9545
Epoch 9/10, Train Loss: 0.1298, Validation Loss: 1.0527, Validation Accuracy: 0.9545
Epoch 10/10, Train Loss: 0.0810, Validation Loss: 1.0923, Validation Accuracy: 0.9545


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model_metrics(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)





In [20]:
#Testing
evaluate_model_metrics(model, test_loader, device)

Accuracy: 0.8182
Precision: 0.6923
Recall: 1.0000
F1 Score: 0.8182
Confusion Matrix:
[[9 4]
 [0 9]]
